# Create ISA-API Investigation from Datascriptor Study Design configuration

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
[missing]

## 1. Setup

Let's import all the required libraries

In [1]:
from time import time
import os
import json

## ISA-API related imports
from isatools.model import Investigation, Study

## ISA-API create mode related imports
from isatools.create.models import StudyDesign
from isatools.create.connectors import generate_study_design_from_config

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

[{'measurement type': 'metabolite profiling', 'technology type': 'NMR spectroscopy', 'protocol type': 'NMR spectroscopy', 'parameter-like file': 'Acquisition Parameter File', 'node name': 'NMR Assay Name', 'key in json_configuration': 'nmr_raw_date_file', 'raw data file': 'Free Induction Decay Data File', 'derived data file': ['Derived Spectral Data File', 'Metabolite Assignment File']}, {'measurement type': 'targeted metabolite profiling', 'technology type': 'NMR spectroscopy', 'protocol type': 'NMR spectroscopy', 'parameter-like file': None, 'node name': 'NMR Assay Name', 'key in json_configuration': 'nmr_raw_date_file', 'raw data file': 'Free Induction Decay Data File', 'derived data file': ['Derived Spectral Data File', 'Metabolite Assignment File']}, {'measurement type': 'untargeted metabolite profiling', 'technology type': 'NMR spectroscopy', 'protocol type': 'NMR spectroscopy', 'parameter-like file': None, 'node name': 'NMR Assay Name', 'key in json_configuration': 'nmr_raw_date

/Users/massi/.pyenv/versions/3.8.2/envs/isa-create-notebook-3.8.2/src/isatools/isatools/create/models.py:973: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  _kwargs = {key: val for key, val in kwargs.items() if key is not 'qc_sample_type'}


In [2]:
## ISA-API create mode related imports
from isatools.create import models
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator

In [3]:
with open(os.path.abspath(os.path.join(
    "config", "study-design-3-repeated-treatment-datascriptor.json"
)), "r") as config_file:
    study_design_config = json.load(config_file)
study_design_config

{'treatmentPlan': {'screen': {'name': 'screen',
   'duration': None,
   'durationUnit': ''},
  'runIn': {'name': 'run-in', 'duration': None, 'durationUnit': ''},
  'washout': {'name': 'washout', 'duration': '5', 'durationUnit': 'days'},
  'followUp': {'name': 'follow-up', 'duration': 60, 'durationUnit': 'days'},
  'treatments': [{'agent': 'ibuprofen',
    'intensity': 8,
    'intensityUnit': 'mg',
    'duration': 10,
    'durationUnit': 'days'},
   {'agent': 'hydroxy',
    'intensity': 5,
    'intensityUnit': 'mg',
    'duration': 10,
    'durationUnit': 'days'},
   {'agent': 'paracetamol',
    'intensity': 10,
    'intensityUnit': 'mg',
    'duration': 10,
    'durationUnit': 'days'}],
  'elementParams': {'agents': [],
   'intensities': [],
   'intensityUnit': '',
   'durations': [],
   'durationUnit': ''}},
 'generatedStudyDesign': {'name': 'test study',
  'description': 'this is the verbose description',
  'type': {'term': 'crossover design',
   'id': 'OBI:0500003',
   'iri': 'http:

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design_from_config` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [4]:
study_design = generate_study_design_from_config(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [5]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(studies=[study])

The generation of the study design took 0.36 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [6]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 0.27 s.


In [7]:
directory = os.path.abspath(os.path.join('output'))
if not os.path.exists(directory):
    os.makedirs(directory)
with open(os.path.abspath(os.path.join('output','isa-investigation-2-arms-nmr-ms.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [8]:
start = time()
isatab.dump(investigation, os.path.abspath(os.path.join('output')))
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

2020-08-20 17:32:00,757 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 270 paths!
2020-08-20 17:32:00,773 [INFO]: isatab.py(write_study_table_files:1330) >> Rendered 270 paths
2020-08-20 17:32:00,778 [INFO]: isatab.py(write_study_table_files:1337) >> Writing 270 rows
2020-08-20 17:32:01,036 [INFO]: isatab.py(_all_end_to_end_paths:1152) >> Found 60 paths!


KeyError: 'Protocol REF.mass spectrometry'

In [ ]:
%debug

> /Users/massi/.pyenv/versions/3.8.2/envs/isa-create-notebook-3.8.2/src/isatools/isatools/isatab.py(1486)write_assay_table_files()
   1484                             node.executes_protocol.name
   1485                         )
-> 1486                         df_dict[olabel][-1] = node.executes_protocol.name
   1487                         oname_label = None
   1488                         if node.executes_protocol.protocol_type:



To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [ ]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [9]:
len(dataframes)

6

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 5 assay files:
* 1 assay file for Mass. Spec. (treatmennt arm, third epoch: surgery)
* 2 assay files for Mass. Spec. (both arms, fourth epoch: follow-up)
* 2 assay files for NMR (both arms, fourth epoch: follow-up)

In [10]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_CELL_treatment_3_ASSAY_GRAPH_001_metabolite-profiling_NMR-spectroscopy.txt'

'a_CELL_control_3_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_treatment_3_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_treatment_2_ASSAY_GRAPH_000_metabolite-profiling_mass-spectrometry.txt'

'a_CELL_control_3_ASSAY_GRAPH_001_metabolite-profiling_NMR-spectroscopy.txt'

We have 8 subjects in the control arm and 24 samples have been collected (3 blood samples per subject during the follow-up epoch)

We have 10 subjects in the control arm and 40 samples have been collected (3 blood samples per subject during the follow-up epoch and 1 liver sample per subject during the surgery epoch)

In [11]:
study_frame = dataframes['s_study_01.txt']
count_control_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'control' in el)])
count_treatment_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'treatment' in el)])
print("There are {} samples in the control arm (i.e. group)".format(count_control_samples))
print("There are {} samples in the treatment arm (i.e. group)".format(count_treatment_samples))

There are 24 samples in the control arm (i.e. group)
There are 40 samples in the treatment arm (i.e. group)


Each control samples is fractioned and 2 labelled extracts are produced (i.e. biological replicates).

```
[
    "labelling",
    {
        "#replicates": 2
    }
]
```

There are 2 possible combinations of Mass Spec Assay specified in our configuration template and 2 techincal replicate are produced for each combinations:

```
[
    "mass spectrometry",
    {
        "#replicates": 2,
        "instrument": [
            "Agilent QTQF 6510"
        ],
        "injection_mode": [
            "FIA",
            "LC"
        ],
        "acquisition_mode": [
            "positive mode"
        ]
    }
]
```

Two output raw spectral files are produced as the result of reach run

```
[
    "raw spectral data file",
    [
        {
            "node_type": "data file",
            "size": 2,
            "is_input_to_next_protocols": false
        }
    ]
]
```

As a total we expect

$$ N_{rows} = (N_{subjects} \times N_{samples}) \times (N_{biorepl} \times N_{combinations} \times N_{techrepl}) = (8 \times 3) \times (2 \times 2 \times 2 \times 2) = 24 \times 16 = 384 $$

Which we can verify for the mass spectrometry assay file of the control group.

In [12]:
dataframes['a_CELL_treatment_3_ASSAY_GRAPH_001_metabolite-profiling_NMR-spectroscopy.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[instrument],Parameter Value[pulse_sequence],Parameter Value[acquisition_mode],Performer.1,Free Induction Decay Data File
0,GRP-treatment.SBJ-001.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_27-18,pellet,nmr spectroscopy,Bruker AvanceII 1 GHz,CPMG,2D 13C-13C NMR,Ellipsis,raw spectral data file_27-26
1,GRP-treatment.SBJ-001.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_27-1,supernatant,nmr spectroscopy,Bruker AvanceII 1 GHz,CPMG,1D 13C NMR,Ellipsis,raw spectral data file_27-3
2,GRP-treatment.SBJ-001.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_27-18,pellet,nmr spectroscopy,Bruker AvanceII 1 GHz,CPMG,1D 13C NMR,Ellipsis,raw spectral data file_27-20
3,GRP-treatment.SBJ-001.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_27-1,supernatant,nmr spectroscopy,Bruker AvanceII 1 GHz,watergate,1D 13C NMR,Ellipsis,raw spectral data file_27-11
4,GRP-treatment.SBJ-001.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_27-18,pellet,nmr spectroscopy,Bruker AvanceII 1 GHz,watergate,1D 13C NMR,Ellipsis,raw spectral data file_27-30
...,...,...,...,...,...,...,...,...,...,...,...
475,GRP-treatment.SBJ-010.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_11-18,pellet,nmr spectroscopy,Bruker AvanceII 1 GHz,CPMG,2D 13C-13C NMR,Ellipsis,raw spectral data file_11-24
476,GRP-treatment.SBJ-010.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_11-18,pellet,nmr spectroscopy,Bruker AvanceII 1 GHz,CPMG,1D 13C NMR,Ellipsis,raw spectral data file_11-22
477,GRP-treatment.SBJ-010.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_11-1,supernatant,nmr spectroscopy,Bruker AvanceII 1 GHz,watergate,2D 13C-13C NMR,Ellipsis,raw spectral data file_11-7
478,GRP-treatment.SBJ-010.CEL-CELL_treatment_3.SMP...,extraction,Ellipsis,extract_11-18,pellet,nmr spectroscopy,Bruker AvanceII 1 GHz,CPMG,2D 13C-13C NMR,Ellipsis,raw spectral data file_11-26
